In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# Copy the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in k_values: 
    model = KMeans(n_clusters=k)
    model.fit(df_market_data_scaled)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k': k_values,
    'Inertia': inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot(x='k', y='Inertia', title='K Values Elbow Curve', frame_width=800, frame_height=400)
elbow_plot

#### Answer the following question: 

**Question:** What is the best value for `K`?

**Answer:** The best value for `K` would be where the elbow starts at 4 with inertia of 79.022.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_segments = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(crypto_segments)

In [ ]:
# Create a copy of the DataFrame
market_data_predictions = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
market_data_predictions['CryptoCluster'] = crypto_segments

# Display sample data
market_data_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
predictions_scatter_plot = market_data_predictions.hvplot.scatter(
    title='Scatter Plot by Crypto Currency Segment - k=4',
    x='price_change_percentage_24h',
    y='price_change_percentage_7d',
    xlabel='24-Hour Price Change Percentage',
    ylabel='7-Day Price Change Percentage',
    by='CryptoCluster',
    hover_cols=['coin_id'],
    frame_width=700,
    frame_height=500
)

predictions_scatter_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
crypto_pca_data = pca.fit_transform(df_market_data_scaled)

# View the first five rows of the DataFrame. 
crypto_pca_data[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
variance = pca.explained_variance_ratio_

print(variance)

In [ ]:
import numpy as np
total_explained_variance = np.sum(pca.explained_variance_ratio_)
total_explained_variance

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [ ]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(crypto_pca_data, columns=['PC1', 'PC2', 'PC3'])

# Copy the crypto names from the original data
df_market_data_pca['coin_id'] = df_market_data_scaled.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index('coin_id')

# Display sample data
df_market_data_pca.head()

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
pca_k_values = list(range(1, 11))

In [ ]:
# Create an empty list to store the inertia values
pca_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in pca_k_values: 
    model = KMeans(n_clusters=k)
    model.fit(df_market_data_pca)
    pca_inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {
    'k': pca_k_values,
    'Inertia': pca_inertia
}
# Create a DataFrame with the data to plot the Elbow curve
df_pca_elbow = pd.DataFrame(pca_elbow_data)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = df_pca_elbow.hvplot(x='k', y='Inertia', title='PCA K Values Elbow Curve', frame_width=800, frame_height=400)
pca_elbow_plot

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for `K` would be where the elbow starts at 4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The interia starts lower than the original data, reading at 49.665.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the PCA data
pca_model.fit(df_market_data_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_segments_pca = pca_model.predict(df_market_data_pca)

# Print the resulting array of cluster values.
print(crypto_segments_pca)

In [ ]:
# Create a copy of the DataFrame with the PCA data
market_data_pca_predictions = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
market_data_pca_predictions['CryptoClusters'] = crypto_segments_pca

# Display sample data
market_data_pca_predictions.head()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_predictions_scatter_plot = market_data_pca_predictions.hvplot.scatter(
    title='Scatter Plot by Crypto Currency Segment Using Primary Clusters - k=4',
    x='PC1',
    y='PC2',
    by='CryptoClusters',
    hover_cols=['coin_id'],
    frame_width=700,
    frame_height=500
)

pca_predictions_scatter_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_plot + pca_elbow_plot

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!
predictions_scatter_plot + pca_predictions_scatter_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact of intertia from the original data features compared to the data using K-Means features is that, the data using K-Means is lower by 29.357. The values were also much closer together making results easier to analyze, while the original data was significantly scattered. 